In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
import io
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings(action='ignore')
import os
import json
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_dir = '/content/drive/MyDrive/data/ethic'
num = 1
with open(os.path.join(data_dir, 'train', f'talksets-train-{num}.json')) as f:
  temp = json.load(f)

In [4]:
temp[1]['sentences'][1]

{'id': '615281c2aab10bca18a4b716-2',
 'speaker': 2,
 'origin_text': '특히 벗방보는 애들은 진짜 거세 시켜야함',
 'text': '특히 벗방보는 애들은 진짜 거세 시켜야함',
 'types': ['HATE', 'CENSURE', 'SEXUAL'],
 'is_immoral': True,
 'intensity': 2.0,
 'intensity_sum': 10,
 'votes': [{'intensity': 'IRRITABLE', 'voter': {'gender': 'MALE', 'age': 20}},
  {'intensity': 'IRRITABLE', 'voter': {'gender': 'MALE', 'age': 50}},
  {'intensity': 'IRRITABLE', 'voter': {'gender': 'FEMALE', 'age': 20}},
  {'intensity': 'IRRITABLE', 'voter': {'gender': 'MALE', 'age': 50}},
  {'intensity': 'IRRITABLE', 'voter': {'gender': 'MALE', 'age': 50}}],
 'frame_id': 51937,
 'mapped_slots': [{'slot': 'V1', 'token': '벗방보다', 'lu_id': 62117},
  {'slot': 'N1', 'token': '애들', 'lu_id': 12711},
  {'slot': 'V2', 'token': '거세시키다', 'lu_id': 73749}]}

In [5]:
sentences = []
labels = []
intensities = []

number = [1, 2, 3, 4, 5]

for num in number:
  with open(os.path.join(data_dir, 'train', f'talksets-train-{num}.json')) as f:
    train_data = json.load(f)
  for data in train_data:
    texts = data['sentences']
    for text in texts:
      sentence = text['text']
      sentense = re.sub('[\t+]', ' ', sentence)
      sentence = re.sub('[\n+]', ' ', sentence)
      sentence = re.sub('[\s+]', ' ', sentence)
      sentence = re.sub('[^ㄱ-ㅎ가-힣ㅏ-ㅣ0-9a-zA-Z .,?!]', '', sentence)
      label = text['is_immoral']
      intensity = text['intensity']

      sentences.append(sentence)
      labels.append(label)
      intensities.append(intensity)

In [6]:
len(sentences), len(labels), len(intensities)

(363154, 363154, 363154)

In [7]:
with open(os.path.join(data_dir, 'train.tsv'), 'wt', encoding='utf-8') as f:
  for (sentence, label, intensity) in zip(sentences, labels, intensities):
    f.write(f'{sentence}\t{label}\t{intensity}\n')

In [8]:
sentences = []
labels = []
intensities = []

with open(os.path.join(data_dir, 'test', f'talksets-train-6.json')) as f:
    train_data = json.load(f)
for data in train_data:
  texts = data['sentences']
  for text in texts:
    sentence = text['text']
    sentense = re.sub('[\t+]', ' ', sentence)
    sentence = re.sub('[\n+]', ' ', sentence)
    sentence = re.sub('[\s+]', ' ', sentence)
    sentence = re.sub('[^ㄱ-ㅎ가-힣ㅏ-ㅣ0-9a-zA-Z .,?!]', '', sentence)
    label = text['is_immoral']
    intensity = text['intensity']

    sentences.append(sentence)
    labels.append(label)
    intensities.append(intensity)

In [9]:
len(sentences), len(labels)

(45215, 45215)

In [10]:
with open(os.path.join(data_dir, 'test.tsv'), 'wt', encoding='utf-8') as f:
  for (sentence, label, intensity) in zip(sentences, labels, intensities):
    f.write(f'{sentence}\t{label}\t{intensity}\n')

In [12]:
train = pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep='\t', header=None)
test = pd.read_csv(os.path.join(data_dir, 'test.tsv'), sep='\t', header=None)

In [13]:
train

,0,1,2
0,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,True,2.0
1,맞아. 사람들이 진짜 개념이없다,True,2.0
2,저렇게는 살지 말아야지,True,2.0
3,그러게 게으르고 멍청한 사람들은 맞아야해,True,2.0
4,인방 보는 남자는 거르는게 맞다,True,1.0
...,...,...,...
363149,뷔응신처럼 죽지도 못하면서 괜히 뛰어들기는,True,1.4
363150,오죽 하면 그러겠나,False,0.0
363151,너 얼굴 보면 한숨 나온다.,True,1.6
363152,신경 끄세요.,False,0.0
